In [1]:
from importlib import reload
import geopandas as gpd

from scripts import preprocessing as pre

from config import Preprocessing
config = reload(Preprocessing).Configuration()

In [2]:
# Read the training area and training polygons
trainingArea = gpd.read_file(config.training_base_dir/config.training_area_fn)
trainingPolygon = gpd.read_file(config.training_base_dir/config.training_polygon_fn)

print(
    f"Read a total of {trainingPolygon.shape[0]} object polygons and "
    "{trainingArea.shape[0]} training areas."
)
print(f"Polygons will be assigned to training areas in the next steps.")

Read a total of 690 object polygons and {trainingArea.shape[0]} training areas.
Polygons will be assigned to training areas in the next steps.


In [3]:
# Check if the training areas and the training polygons have the same crs
if trainingArea.crs != trainingPolygon.crs:
    print("Training area CRS does not match training_polygon CRS")
    # Areas are less in number so conversion should be faster
    targetCRS = trainingPolygon.crs  
    trainingArea = trainingArea.to_crs(targetCRS)
    
print(trainingPolygon.crs)
print(trainingArea.crs)
assert trainingPolygon.crs == trainingArea.crs

epsg:32629
epsg:32629


In [4]:
trainingPolygon

,Id,geometry
0,0,"POLYGON ((802643.808 1326481.218, 802643.802 1..."
1,0,"POLYGON ((802672.328 1326510.973, 802672.325 1..."
2,0,"POLYGON ((802691.866 1326500.160, 802697.368 1..."
3,0,"POLYGON ((802686.182 1326485.855, 802689.112 1..."
4,0,"POLYGON ((802684.196 1326457.986, 802687.715 1..."
...,...,...
685,0,"POLYGON ((797190.205 1330123.505, 797190.513 1..."
686,0,"POLYGON ((797362.781 1330210.071, 797368.717 1..."
687,0,"POLYGON ((797461.167 1330130.547, 797468.355 1..."
688,0,"POLYGON ((797475.533 1330154.933, 797488.583 1..."


In [5]:
trainingArea

,OBJECTID,CID,BUFF_DIST,ORIG_FID,Shape_Leng,Shape_Area,geometry
0,1,0,1000.0,1,8000.0,4000000.0,"POLYGON ((805013.405 1319634.767, 804993.600 1..."
1,2,0,1000.0,2,8000.0,4000000.0,"POLYGON ((812190.703 1329152.777, 812170.288 1..."
2,3,0,1000.0,3,8000.0,4000000.0,"POLYGON ((813915.694 1313172.060, 813895.409 1..."
3,4,0,1000.0,4,8000.0,4000000.0,"POLYGON ((799407.107 1316341.060, 799387.715 1..."
4,5,0,1000.0,5,8000.0,4000000.0,"POLYGON ((824777.183 1338337.048, 824755.800 1..."
5,6,0,1000.0,6,8000.0,4000000.0,"POLYGON ((795373.011 1302042.460, 795354.085 1..."
6,7,0,1000.0,7,8000.0,4000000.0,"POLYGON ((816039.879 1337402.312, 816019.086 1..."
7,8,0,1000.0,8,8000.0,4000000.0,"POLYGON ((794982.292 1308160.752, 794963.303 1..."
8,9,0,1000.0,9,8000.0,4000000.0,"POLYGON ((823037.522 1311360.950, 823016.676 1..."
9,10,0,1000.0,10,8000.0,4000000.0,"POLYGON ((790941.540 1328516.559, 790922.525 1..."


In [6]:
# Assign serial IDs to training areas and polygons
trainingArea["id"] = range(trainingArea.shape[0])
trainingPolygon["id"] = range(trainingPolygon.shape[0])

In [7]:
# areasWithPolygons contains the object polygons and weighted boundaries for each area!
areas_with_polygons = pre.dividePolygonsInTrainingAreas(trainingPolygon, trainingArea)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/364 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [8]:
config = reload(Preprocessing).Configuration()
input_images = pre.read_input_images(
    config.raw_image_base_dir,
    config.raw_image_file_type,
    config.raw_image_suffix,
)
print(f"Found a total of {len(input_images)} pair of raw image(s) to process!")

Found a total of 3 pair of raw image(s) to process!


In [9]:
import rasterio as rio
from shapely.geometry import box
import matplotlib.pyplot as plt 
from pathlib import Path

In [10]:
pre = reload(pre)
config = reload(Preprocessing).Configuration()

In [11]:
# Run the main function for extracting part of ndvi and 
# pan images that overlap with training areas
pre.findOverlap(
    input_images,
    areas_with_polygons,
    config.path_to_write,
    config.extracted_bands_folder,
    config.extracted_annotation_folder,
    config.extracted_boundary_folder,
    config.bands,
)

{'20220105_100150_83_2421_3B_AnalyticMS_SR_harmonized_clip.tif': [0,
  1,
  10,
  12],
 '20220105_112943_72_1058_3B_AnalyticMS_SR_harmonized_clip.tif': [2],
 '20220105_100155_43_2421_3B_AnalyticMS_SR_harmonized_clip.tif': [2, 7, 11]}

In [ ]:
# Display extracted image
sampleImage = "_0.png"
fn = config.path_to_write/config.extracted_ndvi_filename/sampleImage

ndvi_img = Image.open(fn)
pan_img = Image.open(
    fn.replace(config.extracted_ndvi_filename, config.extracted_pan_filename)
)
read_ndvi_img = np.array(ndvi_img)
read_pan_img = np.array(pan_img)
annotation_im = Image.open(
    fn.replace(config.extracted_ndvi_filename, config.extracted_annotation_filename)
)
read_annotation = np.array(annotation_im)
weight_im = Image.open(
    fn.replace(config.extracted_ndvi_filename, config.extracted_boundary_filename)
)
read_weight = np.array(weight_im)
all_images = np.array([read_ndvi_img, read_pan_img, read_annotation, read_weight])

display_images(np.expand_dims(np.transpose(all_images, axes=(1, 2, 0)), axis=0))
# plt.imshow(read_weight)

In [ ]:
len({3, 4, 5, 6, 8, 9, 13, 14})